##### Besluit verbeterpuntjes:
- Als besluit als eerste aan de pagina staat pakt hij het niet door de ". Besluit"
- Soms heet het kopje 'Beslissing' of 'Beslissing op uw bezwaar', 'Besluit op bezwaar' door de tweede ziet hij geen hoofdletter na ". Besluit X"
- Soms wordt de voorgaande zin niet correct afgesloten met een punt, hierdoor herkent de regex het niet. 
- Soms eindigt de zin met een hoofdletter. Voorbeeld: 
    - "separaat bericht. 3597201-1048329-WJZ Besluit Ik bestuit de door u gevraagde informatie nagenoeg geheel openbaar te maken." (nl.mnre1025.2i.2023.74.doc.1)
- Soms heet het kopje 'Deelbesluit' of 'Conclusie'
- 1x gezien dat er helemaal geen kopje Besluit staat in wat voor vorm dan ook, besluit staat in de lopende tekst
- 'Besluit' ziet er zo uit: 'B 1 it' OCR foutje
- Soms staat er bovenin de brief 'Besluit op uw Wob-verzoek', dit wordt gezien als Besluit kopje
    - Misschien zo fixen: \.\sBesluit\s(?!.*(?:op uw Wob-?verzoek))([A-Z]*(.*?\.\s){0,3})

### Spacy modellen:
https://drive.google.com/drive/folders/1AzG_0rQbi4SYRyxSno41R8wCUszkRG-r?usp=sharing

In [1]:
# Imports
import re
import spacy
import string
import json
import numpy as np
import pandas as pd
from spacy import displacy
from spacy.tokens import DocBin
import dateparser
from datetime import datetime, timezone, timedelta
import pickle
from IPython.display import HTML

pd.options.mode.chained_assignment = None

In [2]:
def html(df):
    display(HTML(df.to_html(render_links=True, escape=False)))

In [3]:
# datums als '2 1 juni 2023' omzetten naar '21 juni 2023' en double whitespace weghalen
def join_numbers(s):
    s = ' '.join(s.split())
    return re.sub(r"(\s?\d)(\s)(\d\s)", '\g<1>\g<3>', s)

In [4]:
# drie jaar aan verzoeken, voor grotere besluit dataset
min_3years = pd.read_pickle('../pipe/data/minfinal.pkl.gz', compression='gzip')

# kijken welke reguliere expressies goed werken en een goede bijdrage leveren
besluit_regex6 = r"\.\sBesluit\s([A-Z]*(.*?\.){0,3})"
besluit_regex8 = r"\.\sBesluit:\s([A-Z]*(.*?\.){0,3})"
besluit_regex9 = r"\d\sBesluit\s([A-Z]*(.*?\.){0,3})"
besluit_regex10 = r"\,\sBesluit\s([A-Z]*(.*?\.){0,3})"

besluit_regex6_2 = r"\.\sBesluit\s([A-Z]*(.*?\.\s){0,4})"
besluit_regex8_2 = r"\.\sBesluit:\s([A-Z]*(.*?\.\s){0,4})"
besluit_regex9_2 = r"\d\sBesluit\s([A-Z]*(.*?\.\s){0,4})"
besluit_regex10_2 = r"\,\sBesluit\s([A-Z]*(.*?\.\s){0,4})"

def regex_context_besluit(s):
    if type(s) == float:
        return None
    else:
        search_result = re.search(besluit_regex6_2, join_numbers(s))
        if search_result == None:
            search_result = re.search(besluit_regex8_2, join_numbers(s))
        if search_result == None:
            search_result = re.search(besluit_regex9_2, join_numbers(s))
        if search_result == None:
            search_result = re.search(besluit_regex10_2, join_numbers(s))
        if search_result != None:
            result = search_result.group(1)
            return result
        else:
            return search_result

In [5]:
# besluit context ophalen
min_3years['besluit_context'] = min_3years['foi_bodyTextOCR'].apply(lambda x: regex_context_besluit(x))

# opgehaalde besluiten naar alle rijen van een document uitsmeren
min_3years['besluit_context'] = min_3years['besluit_context'].groupby(level=0).fillna(method='ffill')
min_3years['besluit_context'] = min_3years['besluit_context'].groupby(level=0).fillna(method='bfill')

In [18]:
# een pagina per document, met daaronder hoeveel wel, en hoeveel niet worden opgehaald

min_3years1 = min_3years[min_3years.index.get_level_values(1) == 1]
len(min_3years1[~min_3years1['besluit_context'].isna()]), len(min_3years1[min_3years['besluit_context'].isna()])

C:\Users\Joran\AppData\Local\Temp\ipykernel_12012\3202380397.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(min_3years1[~min_3years1['besluit_context'].isna()]), len(min_3years1[min_3years['besluit_context'].isna()])


(2688, 670)

In [ ]:
# vorige model laden en uitvoeren om een betere verdeling te krijgen om weer op te trainen
nlp = spacy.load("T:/Desktop/UvA/Jaar3/OndragelijkRepo/noclone/Notebooks/Experimenten/data/spacy_custom_models/nl_besluit_1000_lg")

def type_besluit(besluit_text):
    if besluit_text != None:
        doc = nlp(besluit_text)
        hoogste = max(doc.cats, key=doc.cats.get)
        perc = doc.cats[hoogste]
        return hoogste, round(perc*100, 2)
    else: 
        return None, None

min_3years1['besluit_spacy'] = min_3years1['besluit_context'].apply(lambda x: type_besluit(x))
min_3years1[['besluit_type', 'besluit_zekerheid']] = pd.DataFrame(min_3years1['besluit_spacy'].tolist(), index=min_3years1.index)
min_3years1.drop('besluit_spacy', axis=1, inplace=True)

In [21]:
html(min_3years1[['besluit_type', 'besluit_zekerheid', 'besluit_context']].sample(n=3))

,,besluit_type,besluit_zekerheid,besluit_context
foi_documentId,foi_pageNumber,,,
nl.mnre1045.2i.2021.6.misc.1,1,niet aanwezig,99.99,"Ik heb besloten uw verzoek af te wijzen aangezien er geen documenten bij het ministerie zijn met de door u gevraagde informatie. Wijze van openbaarmaking Dit besluit wordt geanonimiseerd op www.rijksoverheid.nl geplaatst. Hoogachtend, Minister van Economische Zaken en Klimaat, namens deze dhr. drs."
nl.mnre1153.2i.2022.340.besluit.1,1,openbaar,98.30,Ik heb besloten de informatie die u vraagt openbaar te maken. Beoordeling van uw verzoek Uw verzoek gaat over publieke informatie die in één of meer documenten staat. Daarom is de Wet open overheid (Woo) van toepassing. Het uitgangspunt van deze wet is dat informatie in principe openbaar is voor iedereen.
nl.mnre1034.2i.2022.103.misc.1,1,niet aanwezig,81.61,"Ik kan uw verzoek niet inwilligen. Er zijn namelijk geen door u gevraagde documenten bij het RVB aangetroffen. Het RVB was niet betrokken bij aangelegenheden rondom de Prinsessan Sophia Albertina. Het RVB is enkel betrokken bij schepen van de VOC, WIC en Admiraliteit en dus niet bij dit Zweedse schip."


In [22]:
# aparte annotatie datasets om een gebalanceerder resultaat te krijgen

niet_aanwezig = min_3years1[min_3years1['besluit_type'] == 'niet aanwezig']
deels_openbaar = min_3years1[min_3years1['besluit_type'] == 'deels openbaar']
openbaar = min_3years1[min_3years1['besluit_type'] == 'openbaar']
afgewezen = min_3years1[min_3years1['besluit_type'] == 'afgewezen']

niet_aanwezig['besluit_context'].to_csv("data/besluittext/txt_files/niet_aanwezig3jaar.txt", header=False, index=False, sep='\n')
deels_openbaar['besluit_context'].to_csv("data/besluittext/txt_files/deels_openbaar3jaar.txt", header=False, index=False, sep='\n')
openbaar['besluit_context'].to_csv("data/besluittext/txt_files/openbaar3jaar.txt", header=False, index=False, sep='\n')
afgewezen['besluit_context'].to_csv("data/besluittext/txt_files/afgewezen3jaar.txt", header=False, index=False, sep='\n')

In [23]:
# exporteer de besluiten in een textfile met per lijn een besluit
# annoteren met doccano (zelf gehost op https://please.knuf.space)

min_3years1['besluit_context'].to_csv("data/besluittext/MASTERTEXT2022.txt", header=False, index=False, sep='\n')

In [24]:
# geannoteerde bestanden inlezen en samenvoegen
besluit_annotated = pd.read_csv('data/besluittext/1000met2022/1000met2022.csv')

besluit_deels_openbaar = besluit_annotated[besluit_annotated['label'] == 'deels openbaar'].sample(n=142)
besluit_afgewezen = pd.read_csv('data/besluittext/gescheiden/besluit_afgewezen.csv')
besluit_niet_aanwezig = pd.read_csv('data/besluittext/gescheiden/besluit_niet_aanwezig.csv')
besluit_openbaar = pd.read_csv('data/besluittext/gescheiden/besluit_openbaar.csv')

besluiten_labeled = pd.concat([besluit_deels_openbaar, besluit_afgewezen, besluit_niet_aanwezig, besluit_openbaar])

# spacy wilde graag een kolom met 'answer:accept' erin
besluiten_labeled.drop('Comments', axis=1, inplace=True)
besluiten_labeled['answer'] = 'accept'

# besluiten shuffelen zodat bij het splitten een representatieve verdeling is
besluiten_shuffled = besluiten_labeled.sample(frac=1)

besluiten_shuffled['label'].value_counts()

label
deels openbaar    293
niet aanwezig     237
afgewezen         190
openbaar          180
Name: count, dtype: int64

In [85]:
# geannoteerde data opdelen in train, dev en test sets (75%, 15%, 10%)

besluit_train = besluiten_shuffled[0:674]
besluit_dev = besluiten_shuffled[675:809]
besluit_test = besluiten_shuffled[810:899]

In [48]:
# geannoteerde data opslaan in een spacy DocBin en het hierna trainen met CLI
# https://catherinebreslin.medium.com/text-classification-with-spacy-3-0-d945e2e8fc44

nlp = spacy.blank("nl")
db = DocBin()

categories = ['openbaar', 'deels openbaar', 'afgewezen', 'niet aanwezig']
for row in besluit_test.iterrows():
    if remove_stopw:
        doc = nlp.make_doc(remove_stopwords(row[1]["text"]))
    else:
        doc = nlp.make_doc(row[1]["text"])
    doc.cats = {category: 0 for category in categories}
    doc.cats[row[1]["label"]] = 1
    db.add(doc)
db.to_disk('data/besluittext/gescheiden/test_balanced_besluit.spacy')

In [93]:
nlp_gescheiden = spacy.load("T:/Desktop/UvA/Jaar3/OndragelijkRepo/noclone/Notebooks/Experimenten/data/spacy_custom_models/nl_besluit_gescheiden_lg")

def type_besluit(besluit_text):
    if besluit_text != None:
        doc = nlp_gescheiden(besluit_text)
        hoogste = max(doc.cats, key=doc.cats.get)
        perc = doc.cats[hoogste]
        return hoogste, round(perc*100, 2)
    else: 
        return None, None
    
min_3years1['besluit_spacy'] = min_3years1['besluit_context'].apply(lambda x: type_besluit(x))
min_3years1[['besluit_type', 'besluit_zekerheid']] = pd.DataFrame(min_3years1['besluit_spacy'].tolist(), index=min_3years1.index)
min_3years1.drop('besluit_spacy', axis=1, inplace=True)

display(min_3years1['besluit_type'].value_counts())
html(min_3years1[['besluit_type', 'besluit_zekerheid', 'besluit_context']].sample(n=15))

besluit_type
deels openbaar    1920
afgewezen          295
niet aanwezig      240
openbaar           233
Name: count, dtype: int64

,,besluit_type,besluit_zekerheid,besluit_context
foi_documentId,foi_pageNumber,,,
nl.mnre1153.2i.2023.483.doc.1,1,None,NaN,None
nl.mnre1013.2i.2022.5.misc.1,1,None,NaN,None
nl.mnre1153.2i.2022.142.misc.1,1,niet aanwezig,100.00,Ik heb besloten uw verzoek af te wijzen aangezien er geen document bij het ministerie is met de door u gevraagde informatie. Uit de door ons verrichte zoekactie naar documenten is gebleken dat er geen melding ten grondslag lag aan de inspectie die op uw bedrijf plaatsgevonden heeft. Het betrof een zogenaamde ‘eigen initiatief inspectie’. Vragen Mocht u vragen hebben over de afhandeling van uw verzoek dan kunt u contact opnemen met de heer M.A.P.
nl.mnre1153.2i.2023.420.doc.1,1,deels openbaar,60.86,"Ik heb besloten de informatie inzake Woo/2022/112 deels openbaar te maken. Waarom ik een deel van de informatie niet openbaar maak, leest u verderop in deze brief. In bijlage 1 vindt u een opsomming van de artikelen uit de Woo die voor mijn besluit van belang zijn. Inzake Woo/2022/122 zijn geen documenten aangetroffen."
nl.mnre1018.2i.2023.36.doc.1,1,deels openbaar,99.97,Ik besluit documenten 1 tot en met 8 deels openbaar te maken. Document 9 is reeds openbaar en kunt u vinden op de website van de rijksoverheid. https: . monumenten.
nl.mnre1073.2i.2022.8.misc.1,1,openbaar,99.57,"Ik maak het document openbaar. Wijze van openbaarmaking Dit besluit zal binnen vijf werkdagen na toezending geanonimiseerd worden gepubliceerd op www.rijksoverheid.nl en is via de volgende link op te zoeken. http://www.rijksoverheid.nl/documenten?trefwoord=&periode-van=&periode- tot=&onderdeel=Ministerie+ van +Sociale+ Zaken +en+Werkgelegenheid&type=Wob-verzoek Datum: 10 februari 2022 Hoogachtend, de Minister van Sociale Zaken en Werkgelegenheid, namens deze, Mevr. mr."
nl.mnre1153.2i.2023.660.doc.1,1,deels openbaar,99.94,"Ik heb besloten de informatie die u vraagt gedeeltelijk openbaar te maken. Waarom ik een deel van de informatie niet openbaar maak, leest u verderop in deze brief."
nl.mnre1018.2i.2022.37.misc.1,1,openbaar,75.73,"Overzicht van personeel binnen de luchtverkeersleiding van de Koninklijke Luchtmacht dat een bindingspremie is toegekend, in relatie tot de gestelde eisen in de nota toekenning bindingspremie; Overzicht van het verloop van opleidingen binnen de luchtverkeersleiding van de Koninklijke Luchtmacht en hoeveel opleidingen zijn gestopt, langer hebben geduurd, zijn uitgestel of onmogelijk zijn geweest door personele onderbezetting. Ik besluit de door u gevraagde informatie openbaar te maken. De informatie is hieronder opgenomen. Ten aanzien van vraag 1 In de periode 2018 tot en met de datum van uw verzoek zijn diverse bindingspremies toegekend."
nl.mnre1013.2i.2023.41.doc.1,1,deels openbaar,100.00,"Uw verzoek wordt gedeeltelijk ingewilligd. Ik heb in mijn beoordeling getracht zo transparant mogelijk te zijn. Er doet zich echter een aantal uitzonderingsgronden en beperkingen voor zoals genoemd in de Woo, die een beperkte kennisname van hetgeen in het document is opgenomen, rechtvaardigen. Voor de motivering hiervan verwijs ik naar het onderdeel ‘Overwegingen’ van dit besluit."


In [87]:
JsonFrame = pd.read_pickle('data/besluittext/JsonFrame.pkl.gz', compression='gzip')
BesluitFrame = JsonFrame[JsonFrame['label_name'] == 'besluit']

In [91]:
nlp_gescheiden = spacy.load("T:/Desktop/UvA/Jaar3/OndragelijkRepo/noclone/Notebooks/Experimenten/data/spacy_custom_models/nl_besluit_gescheiden_lg")

def type_besluit(besluit_text):
    if besluit_text != None:
        doc = nlp_gescheiden(besluit_text)
        hoogste = max(doc.cats, key=doc.cats.get)
        perc = doc.cats[hoogste]
        return hoogste, round(perc*100, 2)
    else: 
        return None, None

In [92]:
BesluitFrame['besluit_spacy_zin'] = BesluitFrame['sentence'].apply(lambda x: type_besluit(x))
BesluitFrame[['besluit_type_zin', 'besluit_zekerheid_zin']] = pd.DataFrame(BesluitFrame['besluit_spacy_zin'].tolist(), index=BesluitFrame.index)
BesluitFrame.drop('besluit_spacy_zin', axis=1, inplace=True)

BesluitFrame['besluit_spacy_text'] = BesluitFrame['text'].apply(lambda x: type_besluit(x))
BesluitFrame[['besluit_type_text', 'besluit_zekerheid_text']] = pd.DataFrame(BesluitFrame['besluit_spacy_text'].tolist(), index=BesluitFrame.index)
BesluitFrame.drop('besluit_spacy_text', axis=1, inplace=True)

getrainde model werkt minder goed op hele teksten, hij is dan ook getrained op alleen de zinnen.

In [103]:
html(BesluitFrame[['besluit_type_zin', 'besluit_type_text', 'besluit_zekerheid_zin', 'besluit_zekerheid_text', 'sentence', 'text']].sample(n=20, random_state=None))

,besluit_type_zin,besluit_type_text,besluit_zekerheid_zin,besluit_zekerheid_text,sentence,text
524,deels openbaar,deels openbaar,100.00,94.11,Ik besluit de door u gevraagde informatie grotendeels openbaar te maken.,""" Ministerie van Volksgezondheid, Welzijn en Sport > Retouradres Postbus 20350 2500 EJ Den Haag Directie Wetgeving en Juridische Zaken Bezoekadres: Parnassusplein 5 2511 VK Den Haag T 070 340 79 11 F 070 340 59 84 www. rijksoverheid.nl Inlichtingen bi Datum: 30 MAMI 2023 Betreft: Besluit op uw Woo-verzoek Ons kenmerk 2022.309 . 3562314-1045831-WIZ Bijtagen In uw e-mail van 30 december 2022, heeft u met een beroep op de Wet open Aen B overheld (hierna: Woo) verzocht om toezending van de jaarverslagen van het Uw mail van Radboudumc met betrekking tot de diensten die zij heeft verricht vanaf 2016. 30 december 2022 Tevens verzoekt u om inzage van de gefactureerde bedragen. Bij brief van 3 januari 2023 is de ontvangst van het verzoek aan u bevestigd en tevens bent u op 9 januari 2023 per e-mail in de gelegenheid gesteld om uw Correspondentie uitsluitend verzoek te verduidelijken. Op 10 januari 2023 heeft u per e-mail uw verzoek richten aan het retouradres verduidelijkt. met vermelding van de datum en het kenmerk van deze brief. In de brief van 30 januari 2023 is de beslistermijn met twee weken verdaagd. Op 23 februari 2022 heeft de behandelaar van uw verzoek telefonisch contact met u opgenomen. Bij e-mailbericht van 23 februari 2023 is gevraagd of u akkoord gaat met het verlengen van de termijn tot uiterlijk 22 maart 2022. U heeft niet met de verlenging van de beslistermijn ingestemd. Op 27 februari 2023 heeft u mijn ministerie in gebreke gesteld wegens het niet tijdig beslissen op uw verzoek. Bij brief van 9 maart 2023 is de ontvangst van uw ingebrekestelling bevestigd. Het Is helaas niet gelukt om op tijd op uw Woo-verzoek te besiissen. Hiervoor bied ik u mijn excuses aan. Hierbij treft u het besluit op uw Woo-verzoek aan. Besluit Ik besluit de door u gevraagde informatie grotendeels openbaar te maken. Ik licht mijn besluit hieronder toe. Inventarisatie documenten Op basis van uw verzoek zijn er meerdere documenten aangetroffen. Deze documenten zijn opgenomen in een inventarislijst, zie hiertoe bijlage B. Belanghebbende(n) Bij de openbaarmaking van diverse documenten spelen de belangen van derden een rol. Deze belanghebbende is vooraf gevraagd om hun mening te geven over de openbaarmaking. Pagina 1 van 7 agina 1 van """
365,deels openbaar,deels openbaar,99.54,99.54,"Ik besluit alle 16 document openbaar te maken, met uitzondering van persoonsgegevens en passages die buiten de reikwijdte van uw verzoek vallen.",""" COR 06.000-100059 Ministerie van Defensie > Retouradres Postbus 20701 2500 ES Den Haag Datum 28- 3-202 Betreft Besluit op uw Woo-verzoek Geachte Op 1 augustus 2022 heeft u een verzoek ingediend als bedoeld in artikel 4.1, eerste lid, van de Wet open overheid (Woo). In uw verzoek vraagt u om informatie over de geleverde ondersteuning door de Koninklijke Marechaussee (hierna: KMar) aan de Libanese Grensagentschappen, in de periode 2019 tot 2021. Specifiek vraagt u om openbaarmaking van de volgende informatie: e Alle rapporten en/of officiële documenten van de door de KMar geleverde ondersteuning aan de Libanese Grensagentschappen. e Alle trainingsmateriaal dat de KMar heeft gebruikt tijdens het ondersteunen van de Libanese Grensagentschappen. e Alle correspondentie tussen de KMar en de International Centre for Migration Policy Development (ICMPD) over de door de KMar geleverde ondersteuning aan de Libanese Grensagentschappen. Inventarisatie documenten In antwoord op uw verzoek is een zoekslag gemaakt naar de door u gevraagde informatie. Deze zoekslag heeft 16 documenten opgeleverd, die inhoudelijk zijn beoordeeld. Deze documenten zijn weergegeven in de inventarislijst die als bijlage bij dit besluit is opgenomen. In dit besluit verwijs ik naar de corresponderende nummers op de inventarislijst. Besluit Ik beslu